In [1]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge BeautifulSoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

In [21]:
from bs4 import BeautifulSoup
import requests
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

getting the data ready with latitudes and longitudes

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)

(210, 3)


In [5]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()
df.shape
#cant get geocoder.google to work so using csv file
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/
df_latlong = pd.read_csv('GeoCord.csv') # Read the csv data
df_latlong.head()

--2020-02-07 19:34:37--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.194, 158.85.108.86, 158.85.108.83
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2020-02-07 19:34:38--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-07 19:34:39--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-07 19:34:39--  https

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
# Create new columns in df_assign
df['Latitude'] = np.nan
df['Longitude'] = np.nan

# For each postcode in df_assign, find corresponding coordinates in df_cord and assign it to df_assigned
for idx in df.index:
    c_idx = df_latlong['Postal Code'] == df.loc[idx, 'PostalCode']
    df.at[idx, 'Latitude'] = df_latlong.loc[c_idx, 'Latitude'].values
    df.at[idx, 'Longitude'] = df_latlong.loc[c_idx, 'Longitude'].values

# Display the results
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [7]:
df.head(50)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [8]:
df.loc[df['PostalCode'] == 'M4T']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.38316


toronto map

In [9]:
geoloc = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geoloc.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

Tmap = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(Tmap)  
    
Tmap

The geograpical coordinates of Toronto are 43.653963, -79.387207.


Now to use foursquare API to analyze locations

In [10]:
CLIENT_ID = 'JNBPA3L1NX0WEOOZHANVXHFVL4LHFLW1IRJM5YPUX5ZOM5JP' # ID
CLIENT_SECRET = '4XAIWLPQ5MH523VQD0SLZSZWHDSYPA0B54ISCPOQETPPIPQC' # Secret
VERSION = '20180605'

For now analyzing Central Toronto region 


In [15]:
CT_latitude = df.loc['M4T']['Latitude']
CT_longitude = df.loc['M4T']['Longitude']

In [16]:
LIMIT = 100 # limit to number of venues
radius = 500 # defining scope
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    CT_latitude, 
    CT_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=JNBPA3L1NX0WEOOZHANVXHFVL4LHFLW1IRJM5YPUX5ZOM5JP&client_secret=4XAIWLPQ5MH523VQD0SLZSZWHDSYPA0B54ISCPOQETPPIPQC&v=20180605&ll=43.6895743,-79.38315990000001&radius=500&limit=100'

In [17]:
result = requests.get(url).json()

In [18]:
# function that will return the category of the venue
def get_type(row):
    try:
        categories = row['categories']
    except:
        categories = row['venue.categories']
        
    if len(categories) == 0:
        return None
    else:
        return categories[0]['name']

In [19]:
venues = result['response']['groups'][0]['items']

flattening JSON file

In [31]:
nearby_venues = json_normalize(venues)
nearby_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet
0,e-0-4db7452da86ed8d46c7117f9-0,0,"[{'summary': 'This spot is popular', 'type': '...",4db7452da86ed8d46c7117f9,Loring-Wyle Parkette,276 St. Clair Avenue East,43.690270,-79.383438,"[{'label': 'display', 'lat': 43.69027042721738...",80,CA,Toronto,ON,Canada,"[276 St. Clair Avenue East, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN
1,e-0-4ee13f43b8f775b3731e5a0c-1,0,"[{'summary': 'This spot is popular', 'type': '...",4ee13f43b8f775b3731e5a0c,Reds,Adelaide,43.692816,-79.384504,"[{'label': 'display', 'lat': 43.6928156790476,...",376,CA,Toronto,ON,Canada,"[Adelaide (York), Toronto ON, Canada]","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],York
2,e-0-4bb7c99253649c741a4746fb-2,0,"[{'summary': 'This spot is popular', 'type': '...",4bb7c99253649c741a4746fb,Moore Park Tennis Club,175 Moore Ave,43.693289,-79.382900,"[{'label': 'display', 'lat': 43.69328930873421...",414,CA,Toronto,ON,Canada,"[175 Moore Ave, Toronto ON, Canada]","[{'id': '4e39a956bd410d7aed40cbc3', 'name': 'T...",0,[],NaN
3,e-0-4ae73286f964a5204aa921e3-3,0,"[{'summary': 'This spot is popular', 'type': '...",4ae73286f964a5204aa921e3,Moorevale Park,Moore Ave.,43.693610,-79.383465,"[{'label': 'display', 'lat': 43.69360987933692...",449,CA,Toronto,ON,Canada,"[Moore Ave. (at Welland Ave.), Toronto ON, Can...","[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",0,[],at Welland Ave.


In [32]:
# firstly filtering columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# Moving on to the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_type, axis=1)

# cleaning columns for better representation
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Loring-Wyle Parkette,Park,43.690270,-79.383438
1,Reds,Restaurant,43.692816,-79.384504
2,Moore Park Tennis Club,Tennis Court,43.693289,-79.382900
3,Moorevale Park,Playground,43.693610,-79.383465


In [34]:
print('Foursquare returned {} venues for Central Toronto'.format(nearby_venues.shape[0]))

Foursquare returned 4 venues for Central Toronto


In [37]:
def NearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # getting url in result
        result = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in result])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
venues = NearbyVenues(names=df['Borough'],latitudes=df['Latitude'],longitudes=df['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [39]:
print(venues.shape)
venues.head()

(2213, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,Scarborough,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


Number of venues per neighbourhood

In [40]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,115,115,115,115,115,115
Downtown Toronto,1296,1296,1296,1296,1296,1296
East Toronto,122,122,122,122,122,122
East York,77,77,77,77,77,77
Etobicoke,74,74,74,74,74,74
Mississauga,12,12,12,12,12,12
North York,242,242,242,242,242,242
Scarborough,84,84,84,84,84,84
West Toronto,173,173,173,173,173,173


In [41]:
print('There are {} unique venue categories in Toronto.'.format(len(venues['Venue Category'].unique())))

There are 266 unique venue categories in Toronto.


In [42]:
dummy = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dummy['Neighborhood'] = venues['Neighborhood'] 

dummy.head()
print(dummy.shape)
groups = dummy.groupby('Neighborhood').mean().reset_index()
groups

(2213, 266)


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017391,...,0.000000,0.008696,0.000000,0.000000,0.008696,0.000000,0.000000,0.000000,0.000000,0.008696
1,Downtown Toronto,0.000000,0.000772,0.000772,0.000772,0.000772,0.001543,0.002315,0.001543,0.013117,...,0.002315,0.011574,0.001543,0.000000,0.004630,0.000000,0.006944,0.000000,0.000772,0.003858
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024590,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008197,0.000000,0.000000,0.016393
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.012987,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004132,0.000000,0.004132,0.000000,0.000000,0.000000,0.000000,0.000000,0.008264,...,0.000000,0.000000,0.004132,0.004132,0.008264,0.000000,0.000000,0.004132,0.008264,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,...,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011561,0.000000,0.000000,0.011561,0.000000,0.005780,0.000000,0.000000,0.017341
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000


In [43]:
num_top_venues = 5

for g in groups['Neighborhood']:
    print("----"+g+"----")
    temp = groups[groups['Neighborhood'] == g].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.05
3            Café  0.05
4             Gym  0.03


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.05
2        Hotel  0.03
3   Restaurant  0.03
4          Bar  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.08
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3             Brewery  0.04
4                Park  0.03


----East York----
                  venue  freq
0           Coffee Shop  0.06
1  Fast Food Restaurant  0.04
2   Sporting Goods Shop  0.04
3          Burger Joint  0.04
4                  Bank  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.09
1        Sandwich Place  0.07
2           Coffee Shop  0.05
3  Fast Food Restaurant  0.04
4                   Gym  0.04


----Mississauga----
                 venue  freq
0                Hotel  0.17
1       

In [45]:
def common_venues(row, num_venues):
    categories = row.iloc[1:]
    categories_sorted = categories.sort_values(ascending=False)
    
    return categories_sorted.index.values[0:num_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create new columns according to the number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = groups['Neighborhood']

for ind in np.arange(groups.shape[0]):
    venues_sorted.iloc[ind, 1:] = common_venues(groups.iloc[ind, :], num_top_venues)

venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Restaurant,Sushi Restaurant,Dessert Shop,Pizza Place,Gym,Clothing Store
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Bar,Seafood Restaurant,Park
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Café,Ice Cream Shop,Park,Pub,Restaurant,Bakery
3,East York,Coffee Shop,Fast Food Restaurant,Pizza Place,Pharmacy,Bank,Park,Sporting Goods Shop,Burger Joint,Indian Restaurant,Supermarket
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Fast Food Restaurant,Gym,Liquor Store,Grocery Store,Café,Beer Store
5,Mississauga,Hotel,Coffee Shop,American Restaurant,Middle Eastern Restaurant,Mediterranean Restaurant,Intersection,Burrito Place,Fried Chicken Joint,Gym,Sandwich Place
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Park,Sandwich Place,Shopping Mall,Grocery Store,Café
7,Scarborough,Chinese Restaurant,Breakfast Spot,Fast Food Restaurant,Bakery,Coffee Shop,Pizza Place,Thai Restaurant,Bank,Pharmacy,Park
8,West Toronto,Bar,Café,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Breakfast Spot,Bakery,Yoga Studio,Gift Shop
9,York,Park,Hockey Arena,Convenience Store,Pizza Place,Caribbean Restaurant,Restaurant,Market,Sandwich Place,Brewery,Fast Food Restaurant


In [47]:
# set number of clusters
clusters = 5

clustering = groups.drop('Neighborhood', 1)

# k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(clustering)

# check cluster labels in the dataframe generated for each row
kmeans.labels_[0:10]

array([3, 2, 2, 3, 3, 0, 3, 4, 2, 1], dtype=int32)

In [48]:
merged = groups

# adding cluster labels
merged['Cluster Labels'] = kmeans.labels_

# merging data to add latitude/longitude for each neighborhood
merged = merged.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017391,...,Coffee Shop,Sandwich Place,Park,Café,Restaurant,Sushi Restaurant,Dessert Shop,Pizza Place,Gym,Clothing Store
1,Downtown Toronto,0.0,0.000772,0.000772,0.000772,0.000772,0.001543,0.002315,0.001543,0.013117,...,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Bar,Seafood Restaurant,Park
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024590,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Café,Ice Cream Shop,Park,Pub,Restaurant,Bakery
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Fast Food Restaurant,Pizza Place,Pharmacy,Bank,Park,Sporting Goods Shop,Burger Joint,Indian Restaurant,Supermarket
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Fast Food Restaurant,Gym,Liquor Store,Grocery Store,Café,Beer Store
